We're about to create and use our own MCP Server and MCP Client!

It's pretty simple, but it's not super-simple. The excitment around MCP is about how easy it is to share and use other MCP Servers - making our own does involve a bit of work.

Let's review some python code made mostly by a hard-working Engineering Team:

accounts.py

In [1]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
from IPython.display import display, Markdown

load_dotenv(override=True)

True

In [5]:
from accounts import Account

In [6]:
account = Account.get("Ed")
account

Account(name='ed', balance=10000.0, strategy='', holdings={}, transactions=[], portfolio_value_time_series=[])

In [7]:
account.buy_shares("AMZN", 3, "Because this bookstore website looks promising")

'Completed. Latest details:\n{"name": "ed", "balance": 9801.604, "strategy": "", "holdings": {"AMZN": 3}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 66.132, "timestamp": "2025-08-18 17:51:33", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-08-18 17:51:33", 9960.604]], "total_portfolio_value": 9960.604, "total_profit_loss": -39.39600000000064}'

In [8]:
account.report()

'{"name": "ed", "balance": 9801.604, "strategy": "", "holdings": {"AMZN": 3}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 66.132, "timestamp": "2025-08-18 17:51:33", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-08-18 17:51:33", 9960.604], ["2025-08-18 17:51:35", 9954.604]], "total_portfolio_value": 9954.604, "total_profit_loss": -45.39600000000064}'

In [9]:
account.list_transactions()

[{'symbol': 'AMZN',
  'quantity': 3,
  'price': 66.132,
  'timestamp': '2025-08-18 17:51:33',
  'rationale': 'Because this bookstore website looks promising'}]

### Now we write an MCP server and use it directly!

In [10]:
# Now let's use our accounts server as an MCP server

params = {"command": "uv", "args": ["run", "accounts_server.py"]}
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as server:
    mcp_tools = await server.list_tools()


In [11]:
mcp_tools

[Tool(name='get_balance', title=None, description='Get the cash balance of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_balanceArguments', 'type': 'object'}, outputSchema={'properties': {'result': {'title': 'Result', 'type': 'number'}}, 'required': ['result'], 'title': 'get_balanceOutput', 'type': 'object'}, annotations=None, meta=None),
 Tool(name='get_holdings', title=None, description='Get the holdings of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_holdingsArguments', 'type': 'object'}, outputSchema={'additionalProperties': {'type': 'integer'}, 'title': 'get_holdingsDictOutput', 'type': 'object'}, annotations=None, meta=None),
 Tool(name='buy_shares', title=None, description=

In [12]:
instructions = "You are able to manage an account for a client, and answer questions about the account."
request = "My name is Ed and my account is under the name Ed. What's my balance and my holdings?"
model = "gpt-4.1-mini"

In [13]:

async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    agent = Agent(name="account_manager", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("account_manager"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))


Hi Ed, your current account balance is $9,801.60. In terms of holdings, you have 3 shares of Amazon (AMZN). Is there anything specific you would like to do with your account?

### Now let's build our own MCP Client

In [14]:
from accounts_client import get_accounts_tools_openai, read_accounts_resource, list_accounts_tools

mcp_tools = await list_accounts_tools()
print(mcp_tools)
openai_tools = await get_accounts_tools_openai()
print(openai_tools)

[Tool(name='get_balance', title=None, description='Get the cash balance of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_balanceArguments', 'type': 'object'}, outputSchema={'properties': {'result': {'title': 'Result', 'type': 'number'}}, 'required': ['result'], 'title': 'get_balanceOutput', 'type': 'object'}, annotations=None, meta=None), Tool(name='get_holdings', title=None, description='Get the holdings of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_holdingsArguments', 'type': 'object'}, outputSchema={'additionalProperties': {'type': 'integer'}, 'title': 'get_holdingsDictOutput', 'type': 'object'}, annotations=None, meta=None), Tool(name='buy_shares', title=None, description="B

In [15]:
request = "My name is Ed and my account is under the name Ed. What's my balance?"

with trace("account_mcp_client"):
    agent = Agent(name="account_manager", instructions=instructions, model=model, tools=openai_tools)
    result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

Ed, your account balance is $9,801.60. If you have any other questions or need assistance with your investments, feel free to ask!

In [16]:
context = await read_accounts_resource("ed")
print(context)

{"name": "ed", "balance": 9801.604, "strategy": "", "holdings": {"AMZN": 3}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 66.132, "timestamp": "2025-08-18 17:51:33", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-08-18 17:51:33", 9960.604], ["2025-08-18 17:51:35", 9954.604], ["2025-08-18 17:53:12", 9804.604]], "total_portfolio_value": 9804.604, "total_profit_loss": -195.39600000000064}


In [17]:
from accounts import Account
Account.get("ed").report()

'{"name": "ed", "balance": 9801.604, "strategy": "", "holdings": {"AMZN": 3}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 66.132, "timestamp": "2025-08-18 17:51:33", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-08-18 17:51:33", 9960.604], ["2025-08-18 17:51:35", 9954.604], ["2025-08-18 17:53:12", 9804.604], ["2025-08-18 17:53:15", 10026.604]], "total_portfolio_value": 10026.604, "total_profit_loss": 26.60399999999936}'